In [1]:
import os

import cv2
import numpy as np
from scipy.cluster.vq import vq, kmeans
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import joblib

## Helper functions

In [2]:
# function to extract descriptors using SIFT or SURF
def extract_descriptors(train_dir, extractor):
    if extractor.lower() not in ['sift', 'surf']:
        raise ValueError("extractor must be either sift or surf")
    if extractor.lower() == 'surf':
        detector = cv2.xfeatures2d.SURF_create()
    else:
        detector = cv2.xfeatures2d.SIFT_create()
    desc = {}
    labels = []
    for dir in os.listdir(train_dir):
        label = dir
        for im in os.listdir(f'{train_dir}/{dir}'):
            img = cv2.imread(f'{train_dir}/{dir}/{im}')
            img = cv2.resize(img, (224,224))
            kp, des = detector.detectAndCompute(img, None)
            desc[im] = des
            labels.append(label)
    return desc, labels 

# stacks arrays vertically
def stack_array(desc):
    desc = list(desc.values())
    descriptors = np.array(desc[0])
    for descriptor in desc[1:]:
        descriptors = np.vstack((descriptors, descriptor))
    return descriptors

# K-means clustering
def perform_kmeans(descriptors, cluster_size):
    vocabulary, _ = kmeans(descriptors, cluster_size, 1)
    return vocabulary

# Returns codebook i.e. bag of visual words via vector quantization
def extract_features(unstacked_descriptors, size, vocabulary):
    desc = list(unstacked_descriptors.values())
    im_features = np.zeros((len(desc), size), "float32")
    for i in range(len(desc)):
        words, distance = vq(desc[i], vocabulary)
        for w in words:
            im_features[i][w] += 1
    return im_features

# Codebooks and train/validation sets for SIFT and SURF

In [3]:
sift_desc, sift_labels = extract_descriptors('./images/train/', extractor = 'sift')

In [4]:
sift_descriptors = stack_array(sift_desc)
sift_vocab = perform_kmeans(sift_descriptors, 800)
sift_im_features = extract_features(sift_desc, 800, sift_vocab)

In [5]:
sift_scaler = StandardScaler().fit(sift_im_features)
sift_bovw = sift_scaler.transform(sift_im_features)

In [6]:
surf_desc, surf_labels = extract_descriptors('./images/train/', extractor = 'surf')
surf_descriptors = stack_array(surf_desc)
surf_vocab = perform_kmeans(surf_descriptors, 800)
surf_im_features = extract_features(surf_desc, 800, surf_vocab)

In [8]:
surf_scaler = StandardScaler().fit(surf_im_features)
surf_bovw = surf_scaler.transform(surf_im_features)

### Saving vocabularies and scalers

In [9]:
np.save('./models/sift_vocab.npy', sift_vocab)
np.save('./models/surf_vocab.npy', surf_vocab)

In [10]:
joblib.dump(sift_scaler, './models/SIFT_SCALER.bin')
joblib.dump(surf_scaler, './models/SURF_SCALER.bin')

['./models/SURF_SCALER.bin']

## Creating validation sets

### SIFT

In [11]:
val_sift_desc, val_sift_labels = extract_descriptors("./images/val/", extractor = 'sift')
val_sift_im_features = extract_features(val_sift_desc, 800, sift_vocab)
val_sift_bovw = sift_scaler.transform(val_sift_im_features)

### SURF

In [12]:
val_surf_desc, val_surf_labels = extract_descriptors("./images/val/", extractor = 'surf')
val_surf_im_features = extract_features(val_surf_desc, 800, surf_vocab)
val_surf_bovw = surf_scaler.transform(val_surf_im_features)

## Grid Search for SIFT SVM

In [13]:
svm_param_grid = [
  {'C': [1, 10, 100, 1000,10000], 'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]
   , 'kernel': ['rbf']},
 ]

In [14]:
sift_svc = SVC()
sift_svm_gridsearch = GridSearchCV(sift_svc, svm_param_grid)
sift_svm_gridsearch.fit(sift_bovw, np.array(sift_labels))

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000, 10000],
                          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [16]:
sift_svm_gridsearch.best_params_

{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}

In [42]:
sift_svm_gridsearch.best_score_

0.9057835550531838

### Validation predictions

In [17]:
sift_svm = sift_svm_gridsearch.best_estimator_
sift_svm_preds = sift_svm.predict(val_sift_bovw)

In [18]:
accuracy_score(val_sift_abels, sift_svm_preds)

0.9594882729211087

In [19]:
joblib.dump(sift_svm, './models/SVM_SIFT.joblib')

['./models/SVM_SIFT.joblib']

## Grid search for SURF SVM


In [20]:
surf_svc = SVC()
surf_svm_gridsearch = GridSearchCV(surf_svc, svm_param_grid)
surf_svm_gridsearch.fit(surf_bovw, np.array(surf_labels))

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000, 10000],
                          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
                          'kernel': ['rbf']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [21]:
surf_svm_gridsearch.best_params_

{'C': 10, 'gamma': 0.0005, 'kernel': 'rbf'}

In [43]:
surf_svm_gridsearch.best_score_

0.9181491464510334

In [22]:
surf_svm = surf_svm_gridsearch.best_estimator_
surf_svm_preds = surf_svm.predict(val_surf_bovw)

In [25]:
accuracy_score(val_surf_labels, surf_svm_preds)

0.9637526652452025

In [26]:
joblib.dump(surf_svm, './models/SVM_SURF.joblib')

['./models/SVM_SURF.joblib']

# SIFT / RF 

In [27]:
rf_param_grid = [{'n_estimators':np.linspace(200, 2000, 10, dtype=int), 'max_depth':np.linspace(10, 100, 10, dtype=int)}]

In [28]:
sift_rf = RandomForestClassifier()
sift_rf_gridsearch = GridSearchCV(sift_rf, rf_param_grid)
sift_rf_gridsearch.fit(sift_bovw, np.array(sift_labels))

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [29]:
sift_rf_gridsearch.best_params_


{'max_depth': 60, 'n_estimators': 800}

In [30]:
sift_rf = sift_rf_gridsearch.best_estimator_
sift_rf_preds = sift_rf.predict(val_sift_bovw)

In [31]:
accuracy_score(val_sift_labels, sift_rf_preds)

0.9637526652452025

In [33]:
joblib.dump(sift_rf, './models/RF_SIFT.joblib')

['./models/RF_SIFT.joblib']

In [37]:
surf_rf = RandomForestClassifier()
surf_rf_gridsearch = GridSearchCV(surf_rf, rf_param_grid)
surf_rf_gridsearch.fit(surf_bovw, np.array(surf_labels))

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [38]:
surf_rf_gridsearch.best_params_

{'max_depth': 40, 'n_estimators': 1000}

In [39]:
surf_rf = surf_rf_gridsearch.best_estimator_
surf_rf_preds = surf_rf.predict(val_surf_bovw)

In [40]:
accuracy_score(val_surf_labels, surf_rf_preds)

0.9445628997867804

In [41]:
joblib.dump(surf_rf, './models/RF_SURF.joblib')

['./models/RF_SURF.joblib']